# Download images with `sentinelsat`


The *Copernicus Open Access Hub* provides complete, free and open access to Sentinel-1, Sentinel-2, Sentinel-3 and Sentinel-5P user products.

`sentinelsat` makes searching, downloading and retrieving the metadata of Sentinel satellite images from the Copernicus Open Access Hub easy.



In [1]:
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from collections import OrderedDict
from pathlib import Path
from datetime import date
import geopandas as gpd
from IPython.display import display

print(f'sentinelsat : {sentinelsat.__version__}')

sentinelsat : 1.0.1


## Set directory

In [2]:
computer_path = '/Volumes/nbdid-sst-lbrat2104/'
grp_letter    = 'N'

# Directory for all work files
work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'

roi_path = f'{work_path}ROI/'

Path(roi_path).mkdir(parents=True, exist_ok=True)

## Set Region of Interest

In [3]:
roi_file      = f'{roi_path}extent_roi_32631.shp'
roi_file_json = f'{roi_path}extent_roi_4326.geojson'

print(f'ROI file : {roi_file}')

# Convert to WGS 84 if needed

roi_gdf = gpd.read_file(roi_file)

if roi_gdf.crs != 'epsg:4326':
    print(f'Old CRS : {roi_gdf.crs}')
    
    roi_gdf = roi_gdf.to_crs('epsg:4326')

    print((f'New CRS : {roi_gdf.crs}'))


# Write ROI file in GeoJSON format

roi_gdf.to_file(roi_file_json,  driver='GeoJSON')


ROI file : /Volumes/nbdid-sst-lbrat2104/GROUP_N/WORK/ROI/extent_roi_32631.shp
Old CRS : epsg:32631
New CRS : epsg:4326


## Authentification

The Copernicus Open Access Hub and probably most Data Hubs require authentication.

You can provide your credentials with **SentinelAPI(your username , your password)**


In [4]:
user     = 'ndeffense'
password = 'lbrat2104'

# Connect to the API

api = SentinelAPI(user, password)

## Sorting & Filtering

In addition to the *search query keywords* sentinelsat allows filtering and sorting of search results before download. To simplify these operations sentinelsat offers the convenience functions `to_geojson()`, `to_dataframe()` and `to_geodataframe()` which return the search results as a GeoJSON object, Pandas DataFrame or a GeoPandas GeoDataFrame, respectively. `to_dataframe()` and `to_geodataframe()` require pandas and geopandas to be installed, respectively.

In this example we query Sentinel-2 scenes over a location and convert the query results to a Pandas DataFrame. The DataFrame is then sorted by cloud cover and ingestion date. We limit the query to first 5 results within our timespan and download them, starting with the least cloudy scene. Filtering can be done with all data types, as long as you pass the id to the download function.

In [22]:
start_date    = date(2018, 8, 25)
end_date      = date(2018, 8, 27)

dates = (start_date, end_date)

platform_name = 'Sentinel-2'

# Sentinel-2
s2_level      = 'Level-1C'
product_type  = 'S2MSI1C'
cloud_cover   = (0,90)
s2_tile       = '31UFS'

# Sentinel-1
polarisation_mode =  'VV VH'

In [23]:
# Search by polygon, time, and SciHub query keywords

footprint = geojson_to_wkt(read_geojson(roi_file_json))

if platform_name == 'Sentinel-2':

    products = api.query(footprint,
                        date = dates,
                        platformname = platform_name,
                        processinglevel = s2_level,
                        producttype = product_type,
                        cloudcoverpercentage = cloud_cover)


# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

# sort and limit to first 5 sorted products

products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
#products_df_sorted = products_df_sorted[['cloudcoverpercentage','ingestiondate']]

products_df = products_df[products_df['title'].str.contains(s2_tile)]

#display(products_df_sorted.columns)


display(products_df)


#products_df_sorted = products_df_sorted[0:1][0:1]

products_df_sorted = products_df_sorted[0:1]['filename']

print(products_df_sorted)


,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier
0e24fb26-bbcf-4af8-af52-51eea7505a2e,S2A_MSIL1C_20180826T104021_N0206_R008_T31UFS_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2018-08-26T10:40:21.024Z, Instrument: MS...",false,2018-08-26 10:40:21.024,2018-08-26 10:40:21.024,2018-08-26 10:40:21.024,2018-08-26 16:57:16.717,...,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20180826T104021_N0206_R008_T31UFS_2...,0e24fb26-bbcf-4af8-af52-51eea7505a2e,S2A_OPER_MSI_L1C_TL_MPS__20180826T125219_A0165...,S2A_OPER_MSI_L1C_TL_MPS__20180826T125219_A0165...,S2A_OPER_MSI_L1C_DS_MPS__20180826T125219_S2018...


71fb445a-3f76-4ff1-a819-4ea4452dfaeb    S2A_MSIL1C_20180826T104021_N0206_R008_T31UES_2...
Name: filename, dtype: object


In [24]:
# download sorted and reduced products

api.download_all(products_df_sorted.index)

#api.download_all(['96b88d4d-216d-4451-a7f5-038296e58034	'])

Fetching archival status: 100%|██████████| 1/1 [00:00<00:00,  1.11 products/s]


ResultTuple(downloaded={}, retrieval_triggered={'71fb445a-3f76-4ff1-a819-4ea4452dfaeb': {'id': '71fb445a-3f76-4ff1-a819-4ea4452dfaeb', 'title': 'S2A_MSIL1C_20180826T104021_N0206_R008_T31UES_20180826T125219', 'size': 174376297, 'md5': '42EF146D051F399B970CBE6EDA6481C9', 'date': datetime.datetime(2018, 8, 26, 10, 40, 21, 24000), 'footprint': 'POLYGON((4.066065964389006 50.45671446498236,4.097963501108383 50.53670821564332,4.156231340156582 50.68263118874896,4.214817201303932 50.82849148370492,4.273792905856817 50.974334620142365,4.333128373837741 51.12016825845845,4.392764835675447 51.266066533716206,4.453096478039404 51.41195894441646,4.465299057889804 51.44131066874662,4.579544021469633 51.44054116582309,4.546436441462649 50.453523278919015,4.066065964389006 50.45671446498236))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('71fb445a-3f76-4ff1-a819-4ea4452dfaeb')/$value", 'Online': False, 'Creation Date': datetime.datetime(2018, 8, 26, 16, 48, 38, 842000), 'Ingestion D

In [ ]:
footprint = geojson_to_wkt(read_geojson('search_polygon.geojson'))

products = api.query(footprint,
                     producttype='SLC',
                     orbitdirection='ASCENDING',
                     limit=10)

api.download_all(products)

In [33]:
tiles = ['33VUC', '33UUB']

query_kwargs = {
        'platformname': platform_name,
        'producttype': product_type,
        'date': dates}

products = OrderedDict()
for tile in tiles:
    kw = query_kwargs.copy()
    kw['tileid'] = tile
    pp = api.query(**kw)
    products.update(pp)

print(products)

OrderedDict()
